# COVID West AfricaVisualization

In [21]:
#Imports
import numpy as np
import pandas as pd
from datetime import datetime,timedelta
import matplotlib.pyplot as plt
import seaborn as sns
import time
import altair as alt
from vega_datasets import data
import os


## I. Load Base csv files

Source files were obtained from John Hopkins Center for Sytems Science and Engineering: https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data
          
Three main files gathering the statistics for each country starting from january 22th, 2020. One file for the confimed cases, another for the dead cases and the last one for the recovered cases

In [22]:
confirmed = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
death = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')

#### Confirmed Cases data

In [23]:
confirmed.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,94,110,110,120,170,174,237,273,281,299
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,174,186,197,212,223,243,259,277,304,333
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,367,409,454,511,584,716,847,986,1171,1251
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,224,267,308,334,370,376,390,428,439,466
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,4,4,5,7,7,7,8,8,8,10


#### Dead Cases data

In [24]:
death.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,4,4,4,4,4,4,4,6,6,7
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,6,8,10,10,11,15,15,16,17,20
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,25,26,29,31,35,44,58,86,105,130
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,3,3,3,6,8,12,14,15,16,17
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,0,0,0,2,2,2,2,2,2,2


#### Recovered Cases data

In [25]:
recovered.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,2,2,2,2,2,5,5,10,10,10
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,17,31,31,33,44,52,67,76,89,99
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,29,29,31,31,37,46,61,61,62,90
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,1,1,1,1,10,10,10,10,16,21
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,0,0,0,0,0,1,1,1,1,2


##  II. Build Dataset for West-Africa countries

We would like to zoom the covid-19 evolution on West Africa countries. We have 15 countries in West-Africa. By merging the  information from the three previous datasets,we have a new dataframe displaying the number of cases for each day and each category. 
Kindly note that the data is updated on a daily basis

In [26]:
#Load countries and their coordinates
countries_oa = pd.read_csv('countries_OA.csv')
countries = countries_oa['countries'].values

In [27]:
confirmed_oa = confirmed[confirmed['Country/Region'].isin(countries)].drop(columns = ['Province/State'])
death_oa = death[death['Country/Region'].isin(countries)].drop(columns = ['Province/State'])
recovered_oa = recovered[recovered['Country/Region'].isin(countries)].drop(columns = ['Province/State'])

In [28]:
df_conf = pd.melt(confirmed_oa,id_vars=["Country/Region", "Lat", "Long"], var_name="Date", value_name="Confirmed_cases")
df_death = pd.melt(death_oa,id_vars=["Country/Region", "Lat", "Long"], var_name="Date", value_name="Dead")
df_recovered = pd.melt(recovered_oa,id_vars=["Country/Region", "Lat", "Long"], var_name="Date", value_name="Recovered")

In [29]:
#Change Date in Datetime format
df_conf['Date'] = pd.to_datetime(df_conf['Date'])
df_death['Date'] = pd.to_datetime(df_death['Date'])
df_recovered['Date'] = pd.to_datetime(df_recovered['Date'])

In [30]:
#Rename Country/Region by Country
df_conf.rename(columns={'Country/Region':'Country'}, inplace=True)
df_death.rename(columns={'Country/Region':'Country'}, inplace=True)
df_recovered.rename(columns={'Country/Region':'Country'}, inplace=True)

In [31]:
##list all the days from the begining till the previous day
date_list = pd.date_range(start = '2020-01-22', end = pd.datetime.today()-timedelta(1)).tolist()

In [32]:
## add all the countries for each date
# for Confirmed_df
for dt in date_list:
    if df_conf[df_conf['Date'] == dt].shape[0] != countries_oa['countries'].shape[0]:
        existing_dep = df_conf[df_conf['Date'] == dt]['Country']
        non_existing_dep = np.setdiff1d(countries_oa['countries'].values,existing_dep)
        for ct in non_existing_dep:
            if dt == '2020-01-22':
                df_conf = df_conf.append({'Country': ct, 'Lat' : countries_oa[countries_oa['countries'] == ct]['latitude'].values[0] , 'Long': countries_oa[countries_oa['countries'] == ct]['longitude'].values[0], 'Date': dt, 'Confirmed_cases': 0}, ignore_index = True)
            else:
                df_conf = df_conf.append({'Country': ct, 'Lat' : countries_oa[countries_oa['countries'] == ct]['latitude'].values[0] , 'Long': countries_oa[countries_oa['countries'] == ct]['longitude'].values[0], 'Date': dt, 'Confirmed_cases': np.nan}, ignore_index = True)
#for death_df    
for dt in date_list:
    if df_death[df_death['Date'] == dt].shape[0] != countries_oa['countries'].shape[0]:
        existing_dep = df_death[df_death['Date'] == dt]['Country']
        non_existing_dep = np.setdiff1d(countries_oa['countries'].values,existing_dep)
        for ct in non_existing_dep:
            if dt == '2020-01-22':
                df_death = df_death.append({'Country': ct, 'Lat' : countries_oa[countries_oa['countries'] == ct]['latitude'].values[0] , 'Long': countries_oa[countries_oa['countries'] == ct]['longitude'].values[0], 'Date': dt, 'Dead': 0}, ignore_index = True)
            else:
                df_death = df_death.append({'Country': ct, 'Lat' : countries_oa[countries_oa['countries'] == ct]['latitude'].values[0] , 'Long': countries_oa[countries_oa['countries'] == ct]['longitude'].values[0], 'Date': dt, 'Dead': np.nan}, ignore_index = True)
#for recovered_df    
for dt in date_list:
    if df_recovered[df_recovered['Date'] == dt].shape[0] != countries_oa['countries'].shape[0]:
        existing_dep = df_recovered[df_recovered['Date'] == dt]['Country']
        non_existing_dep = np.setdiff1d(countries_oa['countries'].values,existing_dep)
        for ct in non_existing_dep:
            if dt == '2020-01-22':
                df_recovered = df_recovered.append({'Country': ct, 'Lat' : countries_oa[countries_oa['countries'] == ct]['latitude'].values[0] , 'Long': countries_oa[countries_oa['countries'] == ct]['longitude'].values[0], 'Date': dt, 'Recovered': 0}, ignore_index = True)
            else:
                df_recovered = df_recovered.append({'Country': ct, 'Lat' : countries_oa[countries_oa['countries'] == ct]['latitude'].values[0] , 'Long': countries_oa[countries_oa['countries'] == ct]['longitude'].values[0], 'Date': dt, 'Recovered': np.nan}, ignore_index = True)
        

In [33]:
#Sort all df.. by date and Countries... they will have same index 
df_conf = df_conf.sort_values(by = ['Date','Country']).reset_index(drop=True)
df_death = df_death.sort_values(by = ['Date','Country']).reset_index(drop=True)
df_recovered = df_recovered.sort_values(by = ['Date','Country']).reset_index(drop=True)

In [34]:
#Buil our dataframe which is a merge of all df
df = pd.merge(df_conf,df_death)
df['Recovered'] = df_recovered['Recovered']

In [35]:
# Clean df by replacing nan values
# Fill Na values. First Date.. put all NA to 0.0
df.loc[df['Date'] == '2020-01-22'] = df.loc[df['Date'] == '2020-01-22'].fillna(0)

In [36]:
countries = countries_oa['countries'].values.tolist()

In [37]:
# For the remaining dates, create a set of DataFrame by country(knowing that we sort it before by date and country_name)
#and then forward_fill
df_list = []
for ct in countries:
    dfi = df[df['Country'] == ct]
    dfi = dfi.ffill()
    df_list.append(dfi)

covid_oa = pd.concat(df_list,ignore_index=True)

In [38]:
covid_oa

,Country,Lat,Long,Date,Confirmed_cases,Dead,Recovered
0,Burkina Faso,12.2383,-1.5616,2020-01-22,0,0,0
1,Burkina Faso,12.2383,-1.5616,2020-01-23,0,0,0
2,Burkina Faso,12.2383,-1.5616,2020-01-24,0,0,0
3,Burkina Faso,12.2383,-1.5616,2020-01-25,0,0,0
4,Burkina Faso,12.2383,-1.5616,2020-01-26,0,0,0
...,...,...,...,...,...,...,...
1105,Togo,8.6195,0.8248,2020-03-31,34,1,10
1106,Togo,8.6195,0.8248,2020-04-01,36,2,10
1107,Togo,8.6195,0.8248,2020-04-02,39,2,17
1108,Togo,8.6195,0.8248,2020-04-03,40,3,17


In [39]:
covid_oa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1110 entries, 0 to 1109
Data columns (total 7 columns):
Country            1110 non-null object
Lat                1110 non-null float64
Long               1110 non-null float64
Date               1110 non-null datetime64[ns]
Confirmed_cases    1110 non-null int64
Dead               1110 non-null int64
Recovered          1110 non-null int64
dtypes: datetime64[ns](1), float64(2), int64(3), object(1)
memory usage: 60.8+ KB


In [40]:
# create daily cases dataset
covid_oa['Daily_case'] = 0
for ct in countries:
    temp = 0
    for dt in date_list:
        first_cases = covid_oa[(covid_oa['Date'] == dt) & (covid_oa['Country'] == ct)]['Confirmed_cases'].values[0]
        ind = covid_oa[(covid_oa['Date'] == dt) & (covid_oa['Country'] == ct)]['Confirmed_cases'].index[0]
        if first_cases != 0:
            covid_oa.loc[ind,'Daily_case'] = first_cases - temp
            temp = first_cases

In [41]:
covid_oa.to_csv('dailycasenumber_oa.csv', index=False)

In [42]:
covid_oa

,Country,Lat,Long,Date,Confirmed_cases,Dead,Recovered,Daily_case
0,Burkina Faso,12.2383,-1.5616,2020-01-22,0,0,0,0
1,Burkina Faso,12.2383,-1.5616,2020-01-23,0,0,0,0
2,Burkina Faso,12.2383,-1.5616,2020-01-24,0,0,0,0
3,Burkina Faso,12.2383,-1.5616,2020-01-25,0,0,0,0
4,Burkina Faso,12.2383,-1.5616,2020-01-26,0,0,0,0
...,...,...,...,...,...,...,...,...
1105,Togo,8.6195,0.8248,2020-03-31,34,1,10,4
1106,Togo,8.6195,0.8248,2020-04-01,36,2,10,2
1107,Togo,8.6195,0.8248,2020-04-02,39,2,17,3
1108,Togo,8.6195,0.8248,2020-04-03,40,3,17,1
